In [1884]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import re
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [1885]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
train_data.index = train_data["PassengerId"]
train_data = train_data.iloc[:,1:]
test_data.index = test_data["PassengerId"]
test_data = test_data.iloc[:,1:]

**Creation of columns Family Names and Title**

In [1886]:
train_data['Titre'] = train_data.Name.map(lambda x : x.split(",")[1].split(".")[0])
train_data['NomFamille'] = train_data.Name.map(lambda x : x.split(",")[0])
train_data = train_data.drop(["Name","Cabin","Ticket"], axis=1)

test_data['Titre'] = test_data.Name.map(lambda x : x.split(",")[1].split(".")[0])
test_data['NomFamille'] = test_data.Name.map(lambda x : x.split(",")[0])
test_data = test_data.drop(["Name","Cabin","Ticket"], axis=1)

In [1887]:
test_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titre,NomFamille
PassengerId,,,,,,,,,
892,3,male,34.5,0,0,7.8292,Q,Mr,Kelly
893,3,female,47.0,1,0,7.0000,S,Mrs,Wilkes
894,2,male,62.0,0,0,9.6875,Q,Mr,Myles
895,3,male,27.0,0,0,8.6625,S,Mr,Wirz
896,3,female,22.0,1,1,12.2875,S,Mrs,Hirvonen
...,...,...,...,...,...,...,...,...,...
1305,3,male,NaN,0,0,8.0500,S,Mr,Spector
1306,1,female,39.0,0,0,108.9000,C,Dona,Oliva y Ocana
1307,3,male,38.5,0,0,7.2500,S,Mr,Saether


In [1888]:
train_data["Titre"].unique()

array([' Mr', ' Mrs', ' Miss', ' Master', ' Don', ' Rev', ' Dr', ' Mme',
       ' Ms', ' Major', ' Lady', ' Sir', ' Mlle', ' Col', ' Capt',
       ' the Countess', ' Jonkheer'], dtype=object)

In [1889]:
test_data["Titre"].unique()

array([' Mr', ' Mrs', ' Miss', ' Master', ' Ms', ' Col', ' Rev', ' Dr',
       ' Dona'], dtype=object)

In [1890]:
vip = ['Don','Sir', 'Major', 'Col', 'Jonkheer', 'Dr', 'Rev']
femmeenfant = ['Miss', 'Mrs', 'Lady', 'Mlle', 'the Countess', 'Ms', 'Mme', 'Dona', 'Master']
for i in range(np.shape(train_data["Titre"])[0]):
    if (train_data["Titre"][i+1].strip() in vip):
        train_data["Titre"][i+1] = "vip"
    elif (train_data["Titre"][i+1].strip() in femmeenfant):
        train_data["Titre"][i+1] = "femmeenfant"
    else :
        train_data["Titre"][i+1] = "autres"
      


In [1891]:
i=892
while (i<1310):
    if (test_data["Titre"][i].strip() in vip):
        test_data["Titre"][i] = "vip"
    elif (test_data["Titre"][i].strip() in femmeenfant):
        test_data["Titre"][i] = "femmeenfant"
    else :
        test_data["Titre"][i] = "autres"  
    i=i+1

In [1892]:
test_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titre,NomFamille
PassengerId,,,,,,,,,
892,3,male,34.5,0,0,7.8292,Q,autres,Kelly
893,3,female,47.0,1,0,7.0000,S,femmeenfant,Wilkes
894,2,male,62.0,0,0,9.6875,Q,autres,Myles
895,3,male,27.0,0,0,8.6625,S,autres,Wirz
896,3,female,22.0,1,1,12.2875,S,femmeenfant,Hirvonen
...,...,...,...,...,...,...,...,...,...
1305,3,male,NaN,0,0,8.0500,S,autres,Spector
1306,1,female,39.0,0,0,108.9000,C,femmeenfant,Oliva y Ocana
1307,3,male,38.5,0,0,7.2500,S,autres,Saether


In [1893]:
test_data["Fare"][test_data[(test_data['Fare'].isnull())].index]=32

In [1894]:
print(test_data.isnull().sum())

Pclass         0
Sex            0
Age           86
SibSp          0
Parch          0
Fare           0
Embarked       0
Titre          0
NomFamille     0
dtype: int64


In [1895]:
train_data["Titre"] = train_data["Titre"].dropna().map({'autres':0,'vip':1,'femmeenfant':2}).astype(int)
train_data["Sex"] = train_data["Sex"].dropna().map({'female':0,'male':1}).astype(int)
train_data["Embarked"] = train_data["Embarked"].dropna().map({'S':0,'C':1,'Q':2}).astype("int")
train_data["Fare"]=train_data["Fare"].astype("int")

test_data["Titre"] = test_data["Titre"].dropna().map({'autres':0,'vip':1,'femmeenfant':2}).astype(int)
test_data["Sex"] = test_data["Sex"].dropna().map({'female':0,'male':1}).astype(int)
test_data["Embarked"] = test_data["Embarked"].dropna().map({'S':0,'C':1,'Q':2}).astype("int")
test_data["Fare"]=test_data["Fare"].astype("int")

**Deleting the Embarqued Nan Values**

In [1896]:
train_data.drop(train_data[(train_data['Embarked'].isnull())].index, inplace=True)
train_data.isnull().sum()

Survived        0
Pclass          0
Sex             0
Age           177
SibSp           0
Parch           0
Fare            0
Embarked        0
Titre           0
NomFamille      0
dtype: int64

In [1897]:
train_data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titre,NomFamille
PassengerId,,,,,,,,,,
1,0,3,1,22.0,1,0,7,0.0,0,Braund
2,1,1,0,38.0,1,0,71,1.0,2,Cumings
3,1,3,0,26.0,0,0,7,0.0,2,Heikkinen
4,1,1,0,35.0,1,0,53,0.0,2,Futrelle
5,0,3,1,35.0,0,0,8,0.0,0,Allen


In [1898]:
from sklearn.linear_model import LinearRegression
df=train_data.copy()
df.drop(df[(df['Age'].isnull())].index, inplace=True)
X = df.loc[:,["Survived","SibSp","Sex","Pclass","Parch","Fare","Embarked","Titre"]]
Y = df["Age"]
myreg = LinearRegression()
myreg = myreg.fit(X,Y)
print('coef dir =',myreg.coef_, 'ord orgine =',myreg.intercept_)
train_data_reg = train_data.copy()
#train_data_reg["Age"]=train_data["Age"].notna()
for i in (train_data_reg[(train_data_reg['Age'].isnull())].index):
    train_data["Age"][i]=myreg.predict(train_data[["Survived","SibSp","Sex","Pclass","Parch","Fare","Embarked","Titre"]])[i-1]
for i in train_data[(train_data['Age'] <0)].index:
    train_data["Age"][i] = 0.5

coef dir = [-4.49202229e+00 -2.42136812e+00 -1.30544201e+01 -7.94620968e+00
  3.63627869e-03 -2.20397353e-02  4.15458733e-01 -8.21428652e+00] ord orgine = 66.45597602185254


In [1899]:
df=test_data.copy()
df.drop(df[(df['Age'].isnull())].index, inplace=True)
X = df.loc[:,["SibSp","Sex","Pclass","Parch","Fare","Embarked","Titre"]]
Y = df["Age"]
myreg = LinearRegression()
myreg = myreg.fit(X,Y)
print('coef dir =',myreg.coef_, 'ord orgine =',myreg.intercept_)
test_data_reg = test_data.copy()
#train_data_reg["Age"]=train_data["Age"].notna()


coef dir = [  0.14894044 -16.48849901  -6.63807727   0.61847965   0.02813842
   1.6622639  -10.50743349] ord orgine = 61.87375233120403


In [1900]:
for i in (test_data_reg[(test_data_reg['Age'].isnull())].index):
    test_data["Age"][i]=myreg.predict(test_data[["SibSp","Sex","Pclass","Parch","Fare","Embarked","Titre"]])[i-1-892]
for i in test_data[(test_data['Age'] <0)].index:
    test_data["Age"][i] = 0.5

In [1901]:
train_data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked      0
Titre         0
NomFamille    0
dtype: int64

In [1902]:
test_data.isnull().sum()

Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked      0
Titre         0
NomFamille    0
dtype: int64

In [1903]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [1904]:
# créer un objet LabelEncoder
le1 = LabelEncoder()
le2 = LabelEncoder()
# encoder la colonne spécifiée
test_data['NomFamille'] = le1.fit_transform(test_data['NomFamille'])
train_data['NomFamille'] = le2.fit_transform(train_data['NomFamille'])
print(test_data)

             Pclass  Sex        Age  SibSp  Parch  Fare  Embarked  Titre  \
PassengerId                                                                
892               3    1  34.500000      0      0     7         2      0   
893               3    0  47.000000      1      0     7         0      2   
894               2    1  62.000000      0      0     9         2      0   
895               3    1  27.000000      0      0     8         0      0   
896               3    0  22.000000      1      1    12         0      2   
...             ...  ...        ...    ...    ...   ...       ...    ...   
1305              3    1  21.141622      0      0     8         0      0   
1306              1    0  39.000000      0      0   108         1      2   
1307              3    1  38.500000      0      0     7         0      0   
1308              3    1  25.667990      0      0     8         0      0   
1309              3    1  25.696129      1      1    22         1      2   

           

In [1905]:
test_data.iloc[:,0:8]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titre
PassengerId,,,,,,,,
892,3,1,34.500000,0,0,7,2,0
893,3,0,47.000000,1,0,7,0,2
894,2,1,62.000000,0,0,9,2,0
895,3,1,27.000000,0,0,8,0,0
896,3,0,22.000000,1,1,12,0,2
...,...,...,...,...,...,...,...,...
1305,3,1,21.141622,0,0,8,0,0
1306,1,0,39.000000,0,0,108,1,2
1307,3,1,38.500000,0,0,7,0,0


In [1906]:
x_train, x_test, y_train, y_test = train_test_split(train_data.iloc[:,1:9], train_data.iloc[:,0], test_size = 0.33)

In [1907]:
modele_regLog = linear_model.LogisticRegression(random_state = 0,solver = 'liblinear', multi_class = 'auto')
modele_regLog.fit(x_train, y_train)
y_pred = modele_regLog.predict(x_test)
pred = modele_regLog.predict(test_data.iloc[:,0:8])
print(pred)
print(accuracy_score(y_test,y_pred)*100)

[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 1 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 1]
87.07482993197279


In [1908]:
res = test_data.iloc[:,:0]
res = res.assign(Survived=pred)
print(res)

             Survived
PassengerId          
892                 0
893                 1
894                 0
895                 0
896                 1
...               ...
1305                0
1306                1
1307                0
1308                0
1309                1

[418 rows x 1 columns]


In [1909]:
res.to_csv('Submission_try_2.csv', sep =',')